In [ ]:
!pip install langchain
!pip install langchain_community
!pip install langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
import getpass
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangChain API key: ")

Enter your LangChain API key: ··········


In [ ]:
%pip install --upgrade --quiet  langchain-openai tiktoken chromadb langchain
import dotenv
dotenv.load_dotenv()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 4.0 MB/s eta 0:00:00
  

True

In [ ]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "True"

# 1-*Stuff*

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI

loader = WebBaseLoader("https://www.refworld.org/legal/legislation/natlegbod/1968/fr/103559")
docs = loader.load()

llm = ChatOpenAI(temperature=0, model_name="gpt-4o")
chain = load_summarize_chain(llm, chain_type="stuff")

result = chain.invoke(docs)

print(result["output_text"])

The document "Tunisie: Loi N° 68-7 du 1968, relative à la condition des étrangers" is a piece of Tunisian legislation enacted on March 8, 1968, concerning the status and conditions of foreigners in Tunisia. It is categorized under legal instruments and pertains to immigration law. The document is sourced from national legislative bodies and is accessible via Refworld, a database managed by the UNHCR. The UNHCR does not endorse the content, and any views expressed are solely those of the author or publisher.


In [ ]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate
prompt_template = """Write a concise summary of the following in french:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")
llm_chain = LLMChain(llm=llm, prompt=prompt)
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

docs = loader.load()
print(stuff_chain.invoke(docs)["output_text"])

La loi tunisienne N° 68-7 du 8 mars 1968 concerne la condition des étrangers en Tunisie. Publiée par les autorités législatives nationales, cette législation traite des aspects liés aux étrangers et à l'immigration. Le document est accessible via Refworld, une plateforme de ressources juridiques et d'informations sur les réfugiés et les apatrides. UNHCR n'est pas responsable du contenu de cette publication et ne l'endosse pas nécessairement.


# 2-*Map-Reduce*

In [ ]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter

llm = ChatOpenAI(temperature=0)

# Map
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes in french
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [ ]:
#methode2
from langchain import hub

map_prompt = hub.pull("rlm/map-prompt")
map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [ ]:
reduce_template = """The following is a set of summaries:
{docs}
Take these and distill them into a final, consolidated summary of the main themes in french.
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

In [ ]:
# Combine documents into a single string for reduction
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,
    document_variable_name="docs"
)

In [ ]:
# Reduce documents
reduce_documents_chain = ReduceDocumentsChain(
    combine_documents_chain=combine_documents_chain,
    collapse_documents_chain=combine_documents_chain,
    token_max=4000,
)

In [ ]:
# Map-Reduce Chain
map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=map_chain,
    reduce_documents_chain=reduce_documents_chain,
    document_variable_name="docs",
    return_intermediate_steps=False,
)

In [ ]:
# Splitting documents
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

In [ ]:
# Execute the Map-Reduce workflow
result = map_reduce_chain.invoke(split_docs)

print(result["output_text"])

The main themes of the documents are related to laws and regulations concerning aliens and immigration in Tunisia, as well as international protection, asylum capacity support, refugee law and policy, training manuals, and UNHCR resources.


# 3-*Refine*

In [ ]:
chain = load_summarize_chain(llm, chain_type="refine")
result = chain.invoke(split_docs)

print(result["output_text"])

The document is the Tunisian Law No. 68-7 of 1968, which relates to the condition of foreigners in Tunisia. It outlines regulations and provisions regarding aliens and immigration law in the country. The document is not a UNHCR publication, and any views expressed are solely those of the author or publisher.


In [ ]:
prompt_template = """Write a concise summary of the following in french:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary in french"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)
result = chain.invoke({"input_documents": split_docs}, return_only_outputs=True)

In [ ]:
print(refine_result["output_text"])

La loi tunisienne N° 68-7 de 1968 concerne la condition des étrangers et a été promulguée le 8 mars 1968. Elle traite des questions liées aux étrangers et à la législation sur l'immigration en Tunisie.


In [ ]:
print("\n\n".join(result["intermediate_steps"][:3]))

La loi tunisienne N° 68-7 de 1968 concerne la condition des étrangers et a été promulguée le 8 mars 1968. Elle traite des questions liées aux étrangers et à la législation sur l'immigration en Tunisie.

La loi tunisienne N° 68-7 de 1968 concerne la condition des étrangers et a été promulguée le 8 mars 1968. Elle traite des questions liées aux étrangers et à la législation sur l'immigration en Tunisie.
